In [1]:
import config
from bs4 import BeautifulSoup as BS
import requests
import mysql.connector
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import *
import time
import re
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise import KNNBasic
from surprise.model_selection import GridSearchCV
from sklearn.metrics.pairwise import cosine_similarity
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import calinski_harabasz_score
from sklearn.decomposition import PCA
import random

In [5]:
df_scaled2 = pd.read_csv('df_scaled2.csv', index_col='Team')

In [6]:
df_scaled2.head()

,ppda_coef,deep,S,CR,CCOP,CCSP,P,DR,IBS,OBS,...,TKL,FC,AW,BR,DW,AKS,CL,PUNCH,LMT,poss
Team,,,,,,,,,,,,,,,,,,,,,
Bayern Munich,-0.898850,2.973806,2.775593,1.611378,1.795907,2.673514,2.370940,2.120765,3.188202,0.987291,...,-2.081226,-1.795271,-0.716088,1.360479,-0.948188,0.407311,-3.083852,-1.680898,-0.596272,2.474804
Borussia Dortmund,1.390868,0.807589,0.118582,-1.716195,-0.305416,-0.128590,1.854117,2.366656,0.630185,-0.790167,...,-1.469764,-2.115407,-1.202503,-0.403610,-0.713429,-1.157638,-0.662533,0.096810,-0.596272,1.379674
RasenBallsport Leipzig,-0.117029,0.509248,0.807436,-0.068055,0.765069,0.934277,-0.233706,0.317571,1.470131,-0.663206,...,1.420784,0.653768,1.188352,1.945090,1.814138,1.581023,-0.321502,-1.147586,0.708380,-0.100780
Bayer Leverkusen,-0.271538,1.067016,1.341650,0.203158,1.102074,-0.901585,1.991632,0.710995,1.412862,0.701628,...,0.401681,-0.866877,-0.386314,2.796366,0.084750,1.189786,-0.543172,-0.614273,-1.248599,1.562196
Borussia M.Gladbach,2.184874,0.405477,0.273222,-0.600050,0.170355,0.257907,0.882059,-0.272566,0.343840,0.066822,...,-1.414177,-1.475135,-1.433344,-1.101041,-2.419341,-1.157638,-1.003564,-1.503127,0.056054,0.467065


In [24]:
df_scaled2.index

Index(['Bayern Munich', 'Borussia Dortmund', 'RasenBallsport Leipzig',
       'Bayer Leverkusen', 'Borussia M.Gladbach', 'Wolfsburg',
       'Eintracht Frankfurt', 'Werder Bremen', 'Hoffenheim',
       'Fortuna Duesseldorf', 'Hertha Berlin', 'Mainz 05', 'Freiburg',
       'Schalke 04', 'Augsburg', 'VfB Stuttgart', 'Hannover 96', 'Nuernberg',
       'Manchester City', 'Liverpool', 'Chelsea', 'Tottenham', 'Arsenal',
       'Manchester United', 'Wolverhampton Wanderers', 'Everton', 'West Ham',
       'Leicester', 'Watford', 'Crystal Palace', 'Newcastle United',
       'Bournemouth', 'Burnley', 'Southampton', 'Brighton', 'Cardiff',
       'Fulham', 'Huddersfield', 'Barcelona', 'Atletico Madrid', 'Real Madrid',
       'Valencia', 'Sevilla', 'Getafe', 'Espanyol', 'Athletic Club',
       'Real Sociedad', 'Alaves', 'Real Betis', 'Eibar', 'Leganes', 'Levante',
       'Villarreal', 'Celta Vigo', 'Real Valladolid', 'Girona', 'SD Huesca',
       'Rayo Vallecano', 'Paris Saint Germain', 'Lille', 'L

In [53]:
df_scaled3 = df_scaled2.reset_index()

In [54]:
df_scaled3.head()

,Team,ppda_coef,deep,S,CR,CCOP,CCSP,P,DR,IBS,...,TKL,FC,AW,BR,DW,AKS,CL,PUNCH,LMT,poss
0,Bayern Munich,-0.898850,2.973806,2.775593,1.611378,1.795907,2.673514,2.370940,2.120765,3.188202,...,-2.081226,-1.795271,-0.716088,1.360479,-0.948188,0.407311,-3.083852,-1.680898,-0.596272,2.474804
1,Borussia Dortmund,1.390868,0.807589,0.118582,-1.716195,-0.305416,-0.128590,1.854117,2.366656,0.630185,...,-1.469764,-2.115407,-1.202503,-0.403610,-0.713429,-1.157638,-0.662533,0.096810,-0.596272,1.379674
2,RasenBallsport Leipzig,-0.117029,0.509248,0.807436,-0.068055,0.765069,0.934277,-0.233706,0.317571,1.470131,...,1.420784,0.653768,1.188352,1.945090,1.814138,1.581023,-0.321502,-1.147586,0.708380,-0.100780
3,Bayer Leverkusen,-0.271538,1.067016,1.341650,0.203158,1.102074,-0.901585,1.991632,0.710995,1.412862,...,0.401681,-0.866877,-0.386314,2.796366,0.084750,1.189786,-0.543172,-0.614273,-1.248599,1.562196
4,Borussia M.Gladbach,2.184874,0.405477,0.273222,-0.600050,0.170355,0.257907,0.882059,-0.272566,0.343840,...,-1.414177,-1.475135,-1.433344,-1.101041,-2.419341,-1.157638,-1.003564,-1.503127,0.056054,0.467065


In [26]:
df1.Home.unique()

array(['Manchester Utd', 'Newcastle Utd', 'Huddersfield', 'Fulham',
       'Watford', 'Bournemouth', 'Wolves', 'Southampton', 'Liverpool',
       'Arsenal', 'Cardiff City', 'West Ham', 'Everton', 'Leicester City',
       'Tottenham', 'Chelsea', 'Burnley', 'Manchester City', 'Brighton',
       'Crystal Palace', 'Marseille', 'Nantes', 'Montpellier', 'Angers',
       'Saint-Étienne', 'Nice', 'Lille', 'Lyon', 'Bordeaux', 'Paris S-G',
       'Reims', 'Guingamp', 'Dijon', 'Rennes', 'Caen', 'Amiens', 'Monaco',
       'Strasbourg', 'Toulouse', 'Nîmes', 'Bayern Munich',
       'Werder Bremen', 'Freiburg', 'Wolfsburg', 'Hertha BSC',
       'Düsseldorf', "M'gladbach", 'Mainz 05', 'Dortmund', 'Hannover 96',
       'Augsburg', 'Hoffenheim', 'Bayer', 'Eint Frankfurt', 'Nürnberg',
       'Stuttgart', 'RB Leipzig', 'Schalke 04', 'Chievo', 'Lazio',
       'Torino', 'Empoli', 'Bologna', 'Parma', 'Sassuolo', 'Atalanta',
       'Juventus', 'Napoli', 'SPAL', 'Inter', 'Cagliari', 'Udinese',
       'Fiorenti

In [28]:
team_dict = {'Manchester Utd': 'Manchester United', 'Newcastle Utd': 'Newcastle United', 'Wolves': 
             'Wolverhampton Wanderers', 'Cardiff City': 'Cardiff', 'Leicester City': 'Leicester', 'Saint-Étienne': 
             'Saint-Etienne', 'Paris S-G': 'Paris Saint Germain', 'Nîmes': 'Nimes', 'Hertha BSC': 'Hertha Berlin', 
             'Düsseldorf': 'Fortuna Duesseldorf', "M'gladbach": 'Borussia M.Gladbach', 'Dortmund':
             'Borussia Dortmund', 'Bayer': 'Bayer Leverkusen', 'Eint Frankfurt': 'Eintracht Frankfurt',
             'Nürnberg': 'Nuernberg', 'Stuttgart': 'VfB Stuttgart', 'RB Leipzig': 'RasenBallsport Leipzig',
             'Parma': 'Parma Calcio 1913', 'SPAL': 'SPAL 2013', 'Milan': 'AC Milan', 'Betis': 'Real Betis',
             'Athletic Bilbao': 'Athletic Club', 'Leganés': 'Leganes', 'Alavés': 'Alaves', 'Atlético Madrid':
             'Atletico Madrid', 'Valladolid': 'Real Valladolid', 'Huesca': 'SD Huesca'}

In [20]:
df1 = pd.read_csv('df_xg.csv', index_col=0)

In [21]:
df1.head()

,Wk,Day,Date,Home,xG,xG.1,Away,Attendance,Venue,Referee,league,home_goals,away_goals
0,1.0,Fri,8/10/18,Manchester Utd,1.6,1.1,Leicester City,74439.0,Old Trafford,Andre Marriner,epl,2,1
1,1.0,Sat,8/11/18,Newcastle Utd,0.9,1.9,Tottenham,51749.0,St. James' Park,Martin Atkinson,epl,1,2
2,1.0,Sat,8/11/18,Huddersfield,0.5,1.9,Chelsea,24121.0,The John Smith's Stadium,Chris Kavanagh,epl,0,3
3,1.0,Sat,8/11/18,Fulham,0.6,0.9,Crystal Palace,24821.0,Craven Cottage,Mike Dean,epl,0,2
4,1.0,Sat,8/11/18,Watford,1.6,0.3,Brighton,20051.0,Vicarage Road Stadium,Jonathan Moss,epl,2,0


In [46]:
# df2 = pd.read_csv('xg_df.csv', index_col=0)

In [43]:
# df2 = df2.set_index('index')

In [44]:
# df2.head()

,xg_home,xg_away,xg_total,xga_home,xga_away,xga_total,xg_dif
index,,,,,,,
Manchester United,31.4,28.1,59.5,20.3,28.0,48.3,11.2
Newcastle United,22.1,16.7,38.8,23.7,29.7,53.4,-14.6
Huddersfield,15.9,14.2,30.1,30.1,32.2,62.3,-32.2
Fulham,22.7,16.7,39.4,27.7,37.8,65.5,-26.1
Watford,23.1,24.8,47.9,27.5,27.6,55.1,-7.2


In [45]:
# df2['index'] = df2['index'].map(team_dict).fillna(df2['index'])

In [47]:
df1['Home'] = df1['Home'].map(team_dict).fillna(df1['Home'])

In [48]:
df1['Away'] = df1['Away'].map(team_dict).fillna(df1['Away'])

In [49]:
df1.head()

,Wk,Day,Date,Home,xG,xG.1,Away,Attendance,Venue,Referee,league,home_goals,away_goals
0,1.0,Fri,8/10/18,Manchester United,1.6,1.1,Leicester,74439.0,Old Trafford,Andre Marriner,epl,2,1
1,1.0,Sat,8/11/18,Newcastle United,0.9,1.9,Tottenham,51749.0,St. James' Park,Martin Atkinson,epl,1,2
2,1.0,Sat,8/11/18,Huddersfield,0.5,1.9,Chelsea,24121.0,The John Smith's Stadium,Chris Kavanagh,epl,0,3
3,1.0,Sat,8/11/18,Fulham,0.6,0.9,Crystal Palace,24821.0,Craven Cottage,Mike Dean,epl,0,2
4,1.0,Sat,8/11/18,Watford,1.6,0.3,Brighton,20051.0,Vicarage Road Stadium,Jonathan Moss,epl,2,0


In [50]:
cols_to_drop = ['Wk', 'Day', 'Date', 'Attendance', 'Venue', 'Referee', 'league']

In [51]:
df1 = df1.drop(columns=cols_to_drop)

In [58]:
df1.shape

(1824, 6)

In [55]:
df_merged1 = pd.merge(df1, df_scaled3, left_on='Home', right_on='Team')

In [59]:
df_merged2 = pd.merge(df_merged1, df_scaled3, left_on='Away', right_on='Team')

In [68]:
df_merged2.Away.value_counts()

Cagliari                  19
Huddersfield              19
Real Madrid               19
Sevilla                   19
Levante                   19
                          ..
Nuernberg                 17
Augsburg                  17
Hertha Berlin             17
RasenBallsport Leipzig    17
Eintracht Frankfurt       17
Name: Away, Length: 98, dtype: int64

In [71]:
#df_merged2= df_merged2.drop(columns=['Team_x', 'Team_y'])

In [79]:
df_merged2.columns

Index(['Home', 'xG', 'xG.1', 'Away', 'home_goals', 'away_goals', 'ppda_coef_x',
       'deep_x', 'S_x', 'CR_x', 'CCOP_x', 'CCSP_x', 'P_x', 'DR_x', 'IBS_x',
       'OBS_x', 'TBOX_x', 'FKCR_x', 'CRN_x', 'CRNCR_x', 'FKS_x', 'ALB_x',
       'ATB_x', 'TOFF_x', 'BCS_x', 'oppda_coef_x', 'deep_allowed_x', 'INT_x',
       'BLK_x', 'TKL_x', 'FC_x', 'AW_x', 'BR_x', 'DW_x', 'AKS_x', 'CL_x',
       'PUNCH_x', 'LMT_x', 'poss_x', 'ppda_coef_y', 'deep_y', 'S_y', 'CR_y',
       'CCOP_y', 'CCSP_y', 'P_y', 'DR_y', 'IBS_y', 'OBS_y', 'TBOX_y', 'FKCR_y',
       'CRN_y', 'CRNCR_y', 'FKS_y', 'ALB_y', 'ATB_y', 'TOFF_y', 'BCS_y',
       'oppda_coef_y', 'deep_allowed_y', 'INT_y', 'BLK_y', 'TKL_y', 'FC_y',
       'AW_y', 'BR_y', 'DW_y', 'AKS_y', 'CL_y', 'PUNCH_y', 'LMT_y', 'poss_y',
       'score'],
      dtype='object')

In [72]:
df_merged2['xG_diff'] = df_merged2['xG'] - df_merged2['xG.1']
df_merged2['ppda_diff'] = df_merged2['ppda_coef_x'] - df_merged2['ppda_coef_y']
df_merged2['deep_diff'] = df_merged2['deep_x'] - df_merged2['deep_y']
df_merged2['S_diff'] = df_merged2['S_x'] - df_merged2['S_y']
df_merged2['CR_diff'] = df_merged2['CR_x'] - df_merged2['CR_y']
df_merged2['CCOP_diff'] = df_merged2['CCOP_x'] - df_merged2['CCOP_y']
df_merged2['CCSP_diff'] = df_merged2['CCSP_x'] - df_merged2['CCSP_y']
df_merged2['P_diff'] = df_merged2['P_x'] - df_merged2['P_y']
df_merged2['DR_diff'] = df_merged2['DR_x'] - df_merged2['DR_y']
df_merged2['IBS_diff'] = df_merged2['IBS_x'] - df_merged2['IBS_y']
df_merged2['OBS_diff'] = df_merged2['OBS_x'] - df_merged2['OBS_y']
df_merged2['TBOX_diff'] = df_merged2['TBOX_x'] - df_merged2['TBOX_y']
df_merged2['FKCR_diff'] = df_merged2['FKCR_x'] - df_merged2['FKCR_y']
df_merged2['CRN_diff'] = df_merged2['CRN_x'] - df_merged2['CRN_y']
df_merged2['CRNCR_diff'] = df_merged2['CRNCR_x'] - df_merged2['CRNCR_y']
df_merged2['']

Index(['Home', 'xG', 'xG.1', 'Away', 'home_goals', 'away_goals', 'ppda_coef_x',
       'deep_x', 'S_x', 'CR_x', 'CCOP_x', 'CCSP_x', 'P_x', 'DR_x', 'IBS_x',
       'OBS_x', 'TBOX_x', 'FKCR_x', 'CRN_x', 'CRNCR_x', 'FKS_x', 'ALB_x',
       'ATB_x', 'TOFF_x', 'BCS_x', 'oppda_coef_x', 'deep_allowed_x', 'INT_x',
       'BLK_x', 'TKL_x', 'FC_x', 'AW_x', 'BR_x', 'DW_x', 'AKS_x', 'CL_x',
       'PUNCH_x', 'LMT_x', 'poss_x', 'ppda_coef_y', 'deep_y', 'S_y', 'CR_y',
       'CCOP_y', 'CCSP_y', 'P_y', 'DR_y', 'IBS_y', 'OBS_y', 'TBOX_y', 'FKCR_y',
       'CRN_y', 'CRNCR_y', 'FKS_y', 'ALB_y', 'ATB_y', 'TOFF_y', 'BCS_y',
       'oppda_coef_y', 'deep_allowed_y', 'INT_y', 'BLK_y', 'TKL_y', 'FC_y',
       'AW_y', 'BR_y', 'DW_y', 'AKS_y', 'CL_y', 'PUNCH_y', 'LMT_y', 'poss_y'],
      dtype='object')

In [76]:
#df_merged2['score'] = df_merged2['home_goals'] - df_merged2['away_goals']
results = df_merged2.drop(['home_goals', 'away_goals', 'Home', 'Away'], 1)
results['winner'] = None
results['winner'][results.score > 0] = 1
results['winner'][results.score < 0] = -1
results['winner'][results.score == 0] = 0
#results = results[results.winner != 0]

/Users/andrewtriola/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/andrewtriola/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/andrewtriola/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [78]:
results.head()

,xG,xG.1,ppda_coef_x,deep_x,S_x,CR_x,CCOP_x,CCSP_x,P_x,DR_x,...,AW_y,BR_y,DW_y,AKS_y,CL_y,PUNCH_y,LMT_y,poss_y,score,winner
0,1.6,1.1,-0.409202,0.963928,0.436742,-0.539109,0.518836,-1.333851,0.650128,-0.152640,...,0.915421,0.2949,1.087622,-0.519304,1.192046,0.171661,1.086043,0.162862,1,1
1,0.3,1.4,0.853560,-0.602876,-0.569533,0.142217,-0.527652,-0.555771,-1.089636,-1.428681,...,0.915421,0.2949,1.087622,-0.519304,1.192046,0.171661,1.086043,0.162862,-2,-1
2,1.7,3.0,0.197249,-1.125143,-1.148141,0.683545,-1.006553,-0.037051,-0.439448,-1.487350,...,0.915421,0.2949,1.087622,-0.519304,1.192046,0.171661,1.086043,0.162862,-3,-1
3,1.7,1.8,1.492186,-0.684117,-0.468906,-0.791106,-0.013276,-0.037051,0.188290,0.331375,...,0.915421,0.2949,1.087622,-0.519304,1.192046,0.171661,1.086043,0.162862,0,0
4,0.8,1.0,0.083612,0.522902,-0.682739,-1.173769,-0.616337,-1.852571,-0.523710,-0.621989,...,0.915421,0.2949,1.087622,-0.519304,1.192046,0.171661,1.086043,0.162862,1,1
